In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from tqdm import tqdm_notebook as tqdm

In [ ]:
## 원본 데이터를 파싱해서 불러옵니다.

train_party = pd.read_csv("Rawdata/train/train_party.csv", parse_dates=["party_start_time", "party_end_time"])
print(train_party.shape)
train_party.head()

# Preprocessing

## 1. 파티 유지 시간 (Total_seconds)

In [ ]:
import time
from datetime import timedelta
from datetime import datetime

In [ ]:
## 파티의 총 유지 시간을 구하고자 합니다.
## 시작 시간과 끝나는 시간의 차이를 구합니다.
## 이 값을 party_time_delta 컬럼에 넣습니다.

train_party["party_time_delta"] = train_party["party_end_time"] - train_party["party_start_time"]
print(train_party.shape)
train_party.head()

In [ ]:
## (주차 - 1) * 7 + 일자는 1주차 1일을 기준으로 상대적으로 며칠인지를 구합니다.
## party_start_re_day, party_end_re_day : reletive value (상대값)을 나타냅니다.
## (끝 일) - (시작 일) = delta (일)
## 이 값을 party_day_delta_int 컬럼에 넣습니다.
## 이 값은 파티 유지 일수를 나타냅니다.

train_party["party_start_re_day"] = (train_party["party_start_week"]-1) * 7 + train_party["party_start_day"]
train_party["party_end_re_day"] = (train_party["party_end_week"]-1) * 7 + train_party["party_end_day"]
train_party["party_day_delta_int"] = train_party["party_end_re_day"] - train_party["party_start_re_day"]

In [ ]:
## party_day_delta의 값들을 int -> timedelta로 변환하는 작업입니다.

## day_delta_list 를 list로 정의합니다.
## 모든 party_day_delta_int 데이터를 Timedelta화 합니다. 
## Timedelta화 한 값들을 day_delta_list에 append 해서, 리스트화 합니다.

day_delta_list = []

for i in tqdm(range(len(train_party))):
    T = pd.Timedelta(days = train_party["party_day_delta_int"][i])
    day_delta_list.append(T)

In [ ]:
## timedelta화 된 데이터를 party_day_delta에 넣습니다.

train_party["party_day_delta"] = day_delta_list

In [ ]:
## timedelta가 7인 데이터를 확인해서 데이터가 문제 없는지 확인합니다.

train_party.loc[train_party["party_day_delta"] == pd.Timedelta(days = 7)]

In [ ]:
## party_day_delta 와 party_time_delta를 더합니다. 
## 이는 party_time_delta에서 - days 가 나오는 것을 상쇄시킵니다.
## 이것을 party_day_time_delta로 정의합니다.

train_party["party_day_time_delta"] = train_party["party_day_delta"] + train_party["party_time_delta"]
train_party["party_day_time_delta"].sort_values(ascending = False).head()

In [ ]:
## party_day_time_delta를 초 단위로 변환하는 작업입니다.

## day_time_delta_list를 list로 정의합니다.
## party_day_time_delta의 값을 total_seconds() 함수를 통해 초단위로 변환합니다.
## 이 값을 day_time_delta_list에 append 합니다.

day_time_delta_list = []

for i in tqdm(range(len(train_party))):
    seconds = train_party["party_day_time_delta"][i].total_seconds()
    day_time_delta_list.append(seconds)

In [ ]:
## day_time_delta_list의 값을 party_day_time_delta_seconds에 넣습니다.

train_party["party_day_time_total_seconds"] = day_time_delta_list

In [ ]:
## 데이터가 잘 들어갔는지 확인합니다.

train_party[["party_start_time", "party_end_time", "party_time_delta", 
           "party_day_delta", "party_day_time_delta", "party_day_time_total_seconds"]].head()

In [ ]:
## party_day_time_delta_seconds의 값을 내림차순으로 조회합니다.

train_party["party_day_time_total_seconds"].sort_values(ascending = False).head()

In [ ]:
## 가장 값이 큰 8105, 4953, 7629번째 데이터를 확인합니다.

train_party.iloc[[8105, 4953, 7629]][["party_start_time", "party_end_time", "party_time_delta", 
                                    "party_day_delta", "party_day_time_delta", "party_day_time_total_seconds"]]

In [ ]:
## 수작업으로 총 시간을 초단위로 구해서 계산한 값이 맞는지 확인합니다.
## 8105는 오전 11시 56분에 시작해서 오전 05시 04분에 끝냈으므로, 적어도 하루 이상 했다는 것을 알 수 있습니다.
## 시간만 두고 볼 때, 17시간 8분 정도를 했습니다.
## day_delta가 7 이므로, 7일 간 파티를 유지했음을 알 수 있습니다.
## 총 유지 시간은 6일 17시간 8분입니다.
## 6 * 24 * 3600 + 17 * 3600 + 8 * 60 = 580080 입니다.
## 따라서 party_day_time_total_seconds 는 정확한 total_seconds를 나타냄을 알 수 있습니다.

6 * 24 * 3600 + 17 * 3600 + 8 * 60

## 2. 파티 시작 시간, 파티 종료 시간

In [ ]:
train_party.head()

In [ ]:
## 파티 시작 시간과 종료 시간에 대해 각각 시간과 분을 나누어 새로운 컬럼에 넣습니다.

train_party["party_start_hour"] = train_party["party_start_time"].dt.hour
train_party["party_start_minute"] = train_party["party_start_time"].dt.minute

train_party["party_end_hour"] = train_party["party_end_time"].dt.hour
train_party["party_end_minute"] = train_party["party_end_time"].dt.minute

In [ ]:
## 생각대로 잘 되었는지 확인합니다.

print(train_party.shape)
train_party[["party_start_time", "party_start_hour", "party_start_minute", 
           "party_end_time", "party_end_hour", "party_end_minute"]].head()

## 3. 파티 참여 횟수 컬럼 만들기

In [ ]:
## 파티 참여 횟수(pivot 시, aggfunc = sum)를 측정하기 위해 모든 값을 1을 줍니다.

train_party["party_count"] = 1
train_party.head()

## 4. acc_id 별로 추출하기

In [ ]:
train_label = pd.read_csv("Rawdata/train/train_label.csv", index_col = None)

In [ ]:
train_label.head()

In [ ]:
## party_start_hour, party_end_hour == 0 인 것을 24로 치환합니다.

train_party.loc[train_party["party_start_hour"] == 0, "party_start_hour"] = 24
train_party.loc[train_party["party_end_hour"] == 0, "party_end_hour"] = 24

In [ ]:
## party_acc_id라는 DataFrame에 acc_id 별로 append 합니다.
## train_label의 acc_id가 포함된 파티를 user

party_acc_id = pd.DataFrame()

for i in tqdm(range(len(train_label))):
    acc_id = train_label.loc[i, "acc_id"]
    user = train_party[train_party["hashed"].str.contains(acc_id)]            
    user["acc_id"] = [acc_id] * len(user)
    party_acc_id = party_acc_id.append(user)

In [ ]:
print(party_acc_id.shape)
party_acc_id.head()

In [ ]:
## hashed를 party_members로 치환합니다.

party_acc_id.rename(columns={'hashed':'party_members'}, inplace=True)

## 5. party_member 수 구하기

In [ ]:
import re

mem = party_acc_id["party_members"]
mem_list = []

In [ ]:
## 정규표현식을 이용해 소속된 파티의 멤버수를 구합니다.

for i in tqdm(range(len(mem))):
    s = mem.iloc[i]
    k = len(re.findall(r'\w+', s))
    mem_list.append(k)

In [ ]:
party_acc_id["member_size"] = mem_list

In [ ]:
party_acc_id.head()

# Feature Engineering

In [ ]:
## users 데이터에서 필요한 컬럼들만 추출합니다.

party = party_acc_id[['acc_id', 'party_start_week', 'party_start_day',
       'party_end_week', 'party_end_day', 'party_start_re_day',
       'party_end_re_day', 'party_day_delta_int', 'party_day_time_total_seconds',
       'party_start_hour', 'party_end_hour', 'party_count', 'member_size']]

In [ ]:
print(party.shape)
party.tail()

## 1. Total

In [ ]:
## "sum", "mean", "std", "max", "min", "coefficient_of_variation"
## total_seconds의 합, 평균, 표준편차, 최댓값, 최솟값, 변동계수를 구합니다.

list = ["sum", "mean", "std", "max", "min"]
df_total = party[["acc_id"]]

for i in list:
    df = "party_" + str(i)
    df = party.pivot_table("party_day_time_total_seconds", index = "acc_id", aggfunc = str(i))
    df = df.reset_index(drop = False)
    df.rename(columns = {df.columns[1] : "party_total_seconds_" + str(i)}, inplace = True)
    df_total = pd.merge(df_total, df, on = "acc_id")
    df_total = df_total.drop_duplicates(subset = "acc_id")

## 변동 계수    
df_total["party_total_coefficient_of_variation"] = df_total["party_total_seconds_std"] / df_total["party_total_seconds_mean"] * 100

print(df_total.shape)
df_total.head()

## 2. party_start_re_day 별  feature (시간 순)

### 2-1. party_total_seconds_by_re_day

In [ ]:
## re_day 별로 party_day_time_total_seconds의 총합

party_total_seconds_by_re_day = party.pivot_table("party_day_time_total_seconds", index = ["acc_id"], columns = ["party_start_re_day"],
                                                  aggfunc = np.sum).fillna(0)

party_total_seconds_by_re_day = party_total_seconds_by_re_day.reset_index(drop = False)
party_total_seconds_by_re_day

## 컬럼 이름 바꾸기
rename = ["acc_id"]

for i in range(1,57):
    col_name = "party_total_seconds_re_day_" + str(i)
    rename.append(col_name)
    
party_total_seconds_by_re_day.columns = rename


## 8주간 총 시간 / 전체 시간, 7 & 8주간 총 시간 / 전체 시간

party_by_re_day = party_total_seconds_by_re_day
party_by_re_day["total_seconds_ratio_8week_total"] = party_total_seconds_by_re_day.iloc[:, 50:57].T.sum() / party_total_seconds_by_re_day.iloc[:,1:57].T.sum()
party_by_re_day["total_seconds_ratio_7&8week_total"] = party_total_seconds_by_re_day.iloc[:, 43:57].T.sum() / party_total_seconds_by_re_day.iloc[:,1:57].T.sum()
party_by_re_day["total_seconds_ratio_6&7&8week_total"] = party_total_seconds_by_re_day.iloc[:,  36:57].T.sum() / party_total_seconds_by_re_day.iloc[:,1:57].T.sum()

print(party_by_re_day.shape)
party_by_re_day.head()

In [ ]:
## 8주간 변동계수, 7 & 8주간 변동계수, 6 & 7 & 8주간 변동계수

party_day = party.pivot_table("party_day_time_total_seconds", index = ["acc_id"], columns = ["party_start_re_day"], aggfunc = np.sum)

## 8주간 변동계수, 7 & 8주간 변동계수, 6 & 7 & 8주간 변동계수
party_by_re_day["party_coef_var_8_week"] = party_day.iloc[:, 50:57].std(axis=1) / party_day.iloc[:, 50:57].mean(axis = 1) *100
party_by_re_day["party_coef_var_7&8_week"] = party_day.iloc[:, 43:57].std(axis=1) / party_day.iloc[:, 43:57].mean(axis = 1) *100
party_by_re_day["party_coef_var_6&7&8_week"] = party_day.iloc[:, 36:57].std(axis=1) / party_day.iloc[:, 36:57].mean(axis = 1) *100


print(party_by_re_day.shape)
party_by_re_day.head()

In [ ]:
## re_day 별로 party_day_time_total_seconds의 평균

party_mean_seconds_re_day = party.pivot_table("party_day_time_total_seconds", index = ["acc_id"], columns = ["party_start_re_day"], aggfunc = np.mean).fillna(0)
party_mean_seconds_re_day = party_mean_seconds_re_day.reset_index(drop = False)


## 컬럼 이름 바꾸기
rename = ["acc_id"]

for i in range(1,57):
    col_name = "party_mean_seconds_re_day_" + str(i)
    rename.append(col_name)
party_mean_seconds_re_day.columns = rename


## 8주간 총 시간 / 전체 시간, 6 & 7 & 8주간 총 시간 / 전체 시간
party_by_re_day = pd.merge(party_total_seconds_by_re_day, party_mean_seconds_re_day, on = "acc_id")
party_by_re_day["mean_seconds_ratio_8week_total"] = party_mean_seconds_re_day.iloc[:, 50:57].T.mean() / party_mean_seconds_re_day.iloc[:,1:57].T.mean()
party_by_re_day["mean_seconds_ratio_7&8week_total"] = party_mean_seconds_re_day.iloc[:, 43:57].T.mean() / party_mean_seconds_re_day.iloc[:,1:57].T.mean()
party_by_re_day["mean_seconds_ratio_6&7&8week_total"] = party_mean_seconds_re_day.iloc[:, 36:57].T.mean() / party_mean_seconds_re_day.iloc[:,1:57].T.mean()

print(party_by_re_day.shape)
party_by_re_day.head()

## 3. party_start_day 별 feature 얻어내기 (요일별)

### 3-1. party_total_seconds_by_day_of_week

In [ ]:
party_total_seconds_by_day = party.pivot_table("party_day_time_total_seconds", index = ["acc_id"], columns = ["party_start_day"], aggfunc = np.sum).fillna(0)

## acc_id를 column으로 변경합니다.
party_total_seconds_by_day = party_total_seconds_by_day.reset_index(drop = False)

## 컬럼 이름을 바꿔줍니다.
rename = ["acc_id"]
for i in range(1,8):
    col_name = "party_total_seconds_by_day_" + str(i)
    rename.append(col_name)
party_total_seconds_by_day.columns = rename

print(party_total_seconds_by_day.shape)
party_total_seconds_by_day.head()

### 3-2. party_mean_by_day_of_week

In [ ]:
df = party.pivot_table("party_day_time_total_seconds", index = ["acc_id"], columns = ["party_start_day"], aggfunc = np.mean).fillna(0)

## acc_id 를 column으로 변경합니다.
df = df.reset_index(drop = False)

## 컬럼 이름을 바꿔줍니다.
rename = ["acc_id"]
for i in range(1,8):
    col_name = "party_mean_seconds_by_day_" + str(i)
    rename.append(col_name)
df.columns = rename

party_mean_seconds_by_day = df

print(party_mean_seconds_by_day.shape)
party_mean_seconds_by_day.head()

In [ ]:
## 합치기

party_by_day_of_week = pd.merge(party_total_seconds_by_day, party_mean_seconds_by_day, on = "acc_id")

print(party_by_day_of_week.shape)
party_by_day_of_week.head()

## 4. 주말 / 평일

In [ ]:
## 1: 수요일, 2: 목요일, 3: 금요일, 4: 토요일, 5: 일요일, 6: 월요일, 7: 화요일

In [ ]:
party["sequence"] = range(len(party))

In [ ]:
## 주말
## 금요일 18시 ~ 일요일 22시를 주말로 가정했습니다.

friday = party.loc[(party["party_start_day"] == 3) & (party["party_start_hour"] > 18)]
saturday = party.loc[(party["party_start_day"] == 4)]
sunday = party.loc[(party["party_start_day"] == 5) & (party["party_start_hour"] < 22)]

## 금, 토, 일 합치기
weekend = pd.concat([friday, saturday, sunday])
weekend.acc_id.values.sort()

In [ ]:
## 주말인 index
weekend_list = weekend.sequence.values.tolist()

In [ ]:
weekend.head()

In [ ]:
## 평일
## 주말을 제외한 파티가 평일 파티입니다.

weekday = party.loc[party["sequence"].isin(weekend_list) == False]

In [ ]:
weekday.head()

In [ ]:
## 평일 + 주말 == 전체

print(weekday.shape[0] + weekend.shape[0], party.shape[0] )

In [ ]:
## 주말 총 파티유지시간 의 합, 평균, 표준편차, 최댓값, 최솟값을 구합니다.

list = ["sum", "mean", "std", "max", "min"]
df_weekend = weekend[["acc_id"]].drop_duplicates()

for i in list:
    df = weekend.pivot_table("party_day_time_total_seconds", index = "acc_id", aggfunc = str(i))
    df = df.reset_index(drop = False)
    df.rename(columns = {df.columns[1] : "weekend_total_seconds_" + str(i)}, inplace = True)
    df_weekend = pd.merge(df_weekend, df, on = "acc_id")

## 변동 계수    
df_weekend["weekend_coefficient_of_variation"] = df_weekend["weekend_total_seconds_std"] / df_weekend["weekend_total_seconds_mean"] * 100

df_weekend.head()

In [ ]:
print(df_weekend.shape)
df_weekend.head()

In [ ]:
## 주중 총 파티유지시간 의 합, 평균, 표준편차, 최댓값, 최솟값을 구합니다.

list = ["sum", "mean", "std", "max", "min"]
df_weekday = weekday[["acc_id"]].drop_duplicates()

for i in list:
    df = weekday.pivot_table("party_day_time_total_seconds", index = "acc_id", aggfunc = str(i))
    df = df.reset_index(drop = False)
    df.rename(columns = {df.columns[1] : "weekday_total_seconds_" + str(i)}, inplace = True)
    df_weekday = pd.merge(df_weekday, df, on = "acc_id")

## 변동 계수    
df_weekday["weekday_coefficient_of_variation"] = df_weekday["weekday_total_seconds_std"] / df_weekday["weekday_total_seconds_mean"] * 100

df_weekday.head()

In [ ]:
print(df_weekend.shape)
df_weekend.tail()

In [ ]:
print(df_weekday.shape)
df_weekday.tail()

In [ ]:
## 주말과 주중 데이터를 merge합니다.

df_weekday_weekend = pd.merge(df_weekday, df_weekend, on = "acc_id", how = "outer").sort_values(by = "acc_id")
df_weekday_weekend.head()

In [ ]:
## std, coefficient_of_variation을 제외하고, nan 값을 채웁니다.

list_na = ['weekday_total_seconds_sum', 'weekday_total_seconds_mean', 
           'weekday_total_seconds_max', 'weekday_total_seconds_min', 
           'weekend_total_seconds_sum', 'weekend_total_seconds_mean',
           'weekend_total_seconds_max', 'weekend_total_seconds_min']

df_weekday_weekend.fillna({x:0 for x in list_na}, inplace=True)

In [ ]:
##  주중 총,평균,최대 시간 / 총,평균,최대 시간 의 비율을 구합니다.

df_weekday_weekend["weekday_total_sum_ratio"] = df_weekday_weekend["weekday_total_seconds_sum"] / (df_weekday_weekend["weekday_total_seconds_sum"] + df_weekday_weekend["weekend_total_seconds_sum"])
df_weekday_weekend["weekday_total_mean_ratio"] = df_weekday_weekend["weekday_total_seconds_mean"] / (df_weekday_weekend["weekday_total_seconds_mean"] + df_weekday_weekend["weekend_total_seconds_mean"])
df_weekday_weekend["weekday_total_max_ratio"] = df_weekday_weekend["weekday_total_seconds_max"] / (df_weekday_weekend["weekday_total_seconds_max"] + df_weekday_weekend["weekend_total_seconds_max"])

In [ ]:
print(df_weekday_weekend.shape)
df_weekday_weekend.head()

## 5. 파티를 마지막에 끝낸 날짜 - 파티를 처음에 결성한 날짜

In [ ]:
## 8주의 기간 내에 파티를 언제 처음 시작했고, 언제 마지막에 했는지 확인합니다.
## 이는 처음 시작한 7일 간의 파티 시간과 마지막 7일 간의 파티 시간의 변화율을 확인하기 위한 밑작업입니다.
## acc_id 별로 끝낸 날짜의 max와 시작 날짜의 min을 피벗 테이블화 하여 데이터 프레임에 담습니다.

df1 = party.pivot_table("party_end_re_day", index = "acc_id", aggfunc = np.max)
df2 = party.pivot_table("party_start_re_day", index = "acc_id", aggfunc = np.min)

In [ ]:
## 두 데이터 프레임을 합칩니다.

df3 = pd.concat([df1, df2], axis = 1)

In [ ]:
## 끝낸 날짜 - 결정 날짜 + 1을 합니다.

df3["party_last_re_day_first_re_day_diff"] = df3["party_end_re_day"] - df3["party_start_re_day"] + 1

In [ ]:
df3 = df3.reset_index(drop = False)

In [ ]:
df3.head()

In [ ]:
party_last_fist_re_day_diff = df3[["acc_id", "party_last_re_day_first_re_day_diff"]]
print(party_last_fist_re_day_diff.shape)
party_last_fist_re_day_diff.head()

In [ ]:
## max-min의 차이가 7보다 큰 것들의 acc_id를 list화 합니다.
## {(마지막 7일간의 party_time 의 평균) - (처음 7일 간의 party_time 의 평균)} /  (party_last_re_day_first_re_day_diff)
## 처음 7일부터 마지막 7일까지의 변화율을 볼 수 있습니다. 

diff_over_7 = df3.loc[df3["party_last_re_day_first_re_day_diff"] > 7, "acc_id"].values.tolist()

ratio_over_7 = []

for i in tqdm(range(len(diff_over_7))):
    acc_id = diff_over_7[i]
    df_over_7 = party[party["acc_id"] == acc_id]
    end_day = df_over_7["party_end_re_day"].max()
    start_day = df_over_7["party_start_re_day"].min()

    end_list1 = df_over_7.loc[df_over_7["party_end_re_day"] == end_day, "party_day_time_total_seconds"].values.tolist()
    end_list2 = df_over_7.loc[df_over_7["party_end_re_day"] == end_day-1, "party_day_time_total_seconds"].values.tolist()
    end_list3 = df_over_7.loc[df_over_7["party_end_re_day"] == end_day-2, "party_day_time_total_seconds"].values.tolist()
    end_list4 = df_over_7.loc[df_over_7["party_end_re_day"] == end_day-3, "party_day_time_total_seconds"].values.tolist()
    end_list5 = df_over_7.loc[df_over_7["party_end_re_day"] == end_day-4, "party_day_time_total_seconds"].values.tolist()
    end_list6 = df_over_7.loc[df_over_7["party_end_re_day"] == end_day-5, "party_day_time_total_seconds"].values.tolist()
    end_list7 = df_over_7.loc[df_over_7["party_end_re_day"] == end_day-6, "party_day_time_total_seconds"].values.tolist()
    end_mean_k = (np.nansum(end_list1) + np.nansum(end_list2) + np.nansum(end_list3) + np.nansum(end_list4) + np.nansum(end_list5) + np.nansum(end_list6) + np.nansum(end_list7)) / 7

    start_list1 = df_over_7.loc[df_over_7["party_start_re_day"] == start_day, "party_day_time_total_seconds"].values.tolist()
    start_list2 = df_over_7.loc[df_over_7["party_start_re_day"] == start_day+1, "party_day_time_total_seconds"].values.tolist()
    start_list3 = df_over_7.loc[df_over_7["party_start_re_day"] == start_day+2, "party_day_time_total_seconds"].values.tolist()
    start_list4 = df_over_7.loc[df_over_7["party_start_re_day"] == start_day+3, "party_day_time_total_seconds"].values.tolist()
    start_list5 = df_over_7.loc[df_over_7["party_start_re_day"] == start_day+4, "party_day_time_total_seconds"].values.tolist()
    start_list6 = df_over_7.loc[df_over_7["party_start_re_day"] == start_day+5, "party_day_time_total_seconds"].values.tolist()
    start_list7 = df_over_7.loc[df_over_7["party_start_re_day"] == start_day+6, "party_day_time_total_seconds"].values.tolist()
    start_mean_k = (np.nansum(start_list1) + np.nansum(start_list2) + np.nansum(start_list3) + np.nansum(start_list4) + np.nansum(start_list5) + np.nansum(start_list6) + np.nansum(start_list7)) / 7

    ratio = (end_mean_k - start_mean_k) / (end_day - start_day + 1)
    ratio_over_7 = ratio_over_7 + [ratio]
    
ratio_over_7

In [ ]:
## max-min의 차이가 7보다 작거나 같은 것들의 acc_id를 list화 합니다.
## 차이가 7보다 작으므로, 앞뒤로 1일의 기울기를 구합니다.
## {(마지막 1일의 party_time 의 평균) - (처음 1일의 party_time 의 평균)} /  (party_last_re_day_first_re_day_diff)
## 처음 7일부터 마지막 7일까지의 변화율을 볼 수 있습니다. 

diff_under_7 = df3.loc[df3["party_last_re_day_first_re_day_diff"] <= 7, "acc_id"].values.tolist()

ratio_under_7 = []

for i in tqdm(range(len(diff_under_7))):
    acc_id = diff_under_7[i]
    df_under_7 = party[party["acc_id"] == acc_id]
    end_day = df_under_7["party_end_re_day"].max()
    start_day = df_under_7["party_start_re_day"].min()

    end_list1 = df_under_7.loc[df_under_7["party_end_re_day"] == end_day, "party_day_time_total_seconds"].values.tolist()
    end_mean_k = np.mean(end_list1)

    start_list1 = df_under_7.loc[df_under_7["party_start_re_day"] == start_day, "party_day_time_total_seconds"].values.tolist()
    start_mean_k = np.mean(start_list1) 

    ratio = (end_mean_k - start_mean_k) / (end_day - start_day + 1)
    ratio_under_7 = ratio_under_7 + [ratio]
    
ratio_under_7

In [ ]:
## 두 컬럼을 하나의 데이터 프레임으로 합칩니다.

ratio_re_day_over = pd.DataFrame(columns = ["acc_id", "diff_over_7_ratio"])
ratio_re_day_under = pd.DataFrame(columns = ["acc_id", "diff_under_7_ratio"])

In [ ]:
ratio_re_day_over["acc_id"] = diff_over_7 
ratio_re_day_over["diff_over_7_ratio"] = ratio_over_7
ratio_re_day_under["acc_id"] = diff_under_7
ratio_re_day_under["diff_under_7_ratio"] =  ratio_under_7

In [ ]:
ratio_re_day = pd.merge(ratio_re_day_over, ratio_re_day_under, on = "acc_id", how = "outer")
print(ratio_re_day.shape)
ratio_re_day.head()

In [ ]:
party_re_day_diff_ratio = pd.merge(party_last_fist_re_day_diff, ratio_re_day, on = "acc_id")
print(party_re_day_diff_ratio.shape)
party_re_day_diff_ratio.head()

## 6. Total_seconds 구간 나누기

In [ ]:
## 총 시간을 10개의 구간으로 나누어 줍니다.

party.loc[party["party_day_time_total_seconds"] <= 40, "total_seconds_class"] = "time_class_1"
party.loc[(party["party_day_time_total_seconds"] > 40) & (party["party_day_time_total_seconds"] <= 400), "total_seconds_class"] = "time_class_2"
party.loc[(party["party_day_time_total_seconds"] > 400) & (party["party_day_time_total_seconds"] <= 810), "total_seconds_class"] = "time_class_3"
party.loc[(party["party_day_time_total_seconds"] > 810) & (party["party_day_time_total_seconds"] <= 940), "total_seconds_class"] = "time_class_4"
party.loc[(party["party_day_time_total_seconds"] > 940) & (party["party_day_time_total_seconds"] <= 1120), "total_seconds_class"] = "time_class_5"
party.loc[(party["party_day_time_total_seconds"] > 1120) & (party["party_day_time_total_seconds"] <= 1620), "total_seconds_class"] = "time_class_6"
party.loc[(party["party_day_time_total_seconds"] > 1620) & (party["party_day_time_total_seconds"] <= 3600), "total_seconds_class"] = "time_class_7"
party.loc[(party["party_day_time_total_seconds"] > 3600) & (party["party_day_time_total_seconds"] <= 10800), "total_seconds_class"] = "time_class_8"
party.loc[(party["party_day_time_total_seconds"] > 10800) & (party["party_day_time_total_seconds"] <= 43200), "total_seconds_class"] = "time_class_9"
party.loc[(party["party_day_time_total_seconds"] > 43200), "total_seconds_class"] = "time_class_10"

In [ ]:
## time_class 별 count
party_count_by_time_class = party.pivot_table("party_count", index = "acc_id", columns = ["total_seconds_class"], aggfunc = np.sum)

## 컬럼 순서 변경
columns = [ "time_class_1", "time_class_2", "time_class_3", "time_class_4", "time_class_5", 
           "time_class_6", "time_class_7", "time_class_8", "time_class_9", "time_class_10"]

party_count_by_time_class = party_count_by_time_class[columns]

## 컬럼 이름 변경
column_names = ["party_count_by_time_class_1", "party_count_by_time_class_2", "party_count_by_time_class_3", "party_count_by_time_class_4", "party_count_by_time_class_5",
 "party_count_by_time_class_6", "party_count_by_time_class_7", "party_count_by_time_class_8", "party_count_by_time_class_9", "party_count_by_time_class_10"]

party_count_by_time_class.columns = column_names

## NaN 값 채우기
party_count_by_time_class = party_count_by_time_class.fillna(0)

## acc_id를 column으로 변경합니다.
party_count_by_time_class = party_count_by_time_class.reset_index(drop = False)

print(party_count_by_time_class.shape)
party_count_by_time_class.head()

In [ ]:
## time_class 별 total_seconds
party_seconds_by_time_class = party.pivot_table("party_day_time_total_seconds", index = "acc_id", columns = ["total_seconds_class"], aggfunc = np.sum)

## 컬럼 순서 변경
columns = [ "time_class_1", "time_class_2", "time_class_3", "time_class_4", "time_class_5", 
           "time_class_6", "time_class_7", "time_class_8", "time_class_9", "time_class_10"]

party_seconds_by_time_class = party_seconds_by_time_class[columns]

## 컴럼 이름 변경

column_names = ["party_seconds_by_time_class_1", "party_seconds_by_time_class_2", "party_seconds_by_time_class_3", "party_seconds_by_time_class_4", "party_seconds_by_time_class_5",
                "party_seconds_by_time_class_6", "party_seconds_by_time_class_7", "party_seconds_by_time_class_8", "party_seconds_by_time_class_9", "party_seconds_by_time_class_10"]
                                                           
party_seconds_by_time_class.columns = column_names

## NaN 값 채우기
party_seconds_by_time_class = party_seconds_by_time_class.fillna(0)

## acc_id를 column으로 변경합니다.
party_seconds_by_time_class = party_seconds_by_time_class.reset_index(drop = False)

print(party_seconds_by_time_class.shape)
party_seconds_by_time_class.head()

In [ ]:
## time_class 별 mean
party_mean_by_time_class = party.pivot_table("party_day_time_total_seconds", index = "acc_id", columns = ["total_seconds_class"], aggfunc = np.mean)

## 컬럼 순서 변경
columns = [ "time_class_1", "time_class_2", "time_class_3", "time_class_4", "time_class_5", 
           "time_class_6", "time_class_7", "time_class_8", "time_class_9", "time_class_10"]

party_mean_by_time_class = party_mean_by_time_class[columns]

## 컬럼 이름 변경

column_names = ["party_mean_by_time_class_1", "party_mean_by_time_class_2", "party_mean_by_time_class_3", "party_mean_by_time_class_4", "party_mean_by_time_class_5",
                "party_mean_by_time_class_6", "party_mean_by_time_class_7", "party_mean_by_time_class_8", "party_mean_by_time_class_9", "party_mean_by_time_class_10"]
                                                           
party_mean_by_time_class.columns = column_names


## NaN 값 채우기
party_mean_by_time_class = party_mean_by_time_class.fillna(0)

## acc_id 를 column으로 변경합니다.
party_mean_by_time_class = party_mean_by_time_class.reset_index(drop = False)

print(party_mean_by_time_class.shape)
party_mean_by_time_class.head()

## 7. By hour

In [ ]:
party.head()

### 7-1. 시간대별 총 파티 유지 시간

In [ ]:
## 1시부터 24시까지 시간대별 총 파티 유지 시간을 구합니다.

party_total_seconds_by_hour = party.pivot_table(values = "party_day_time_total_seconds", index = "acc_id", columns = "party_start_hour", aggfunc = np.sum)
party_total_seconds_by_hour = party_total_seconds_by_hour.reset_index(drop = False)
party_total_seconds_by_hour = party_total_seconds_by_hour.fillna(0)

## 컬럼 이름을 바꿔줍니다.

rename = ["acc_id"]
for i in range(1,25):
    col_name = "party_total_seconds_by_hour_" + str(i)
    rename.append(col_name)
party_total_seconds_by_hour.columns = rename

print(party_total_seconds_by_hour.shape)
party_total_seconds_by_hour.head()

### 7-2. 시간대별 평균 파티 유지 시간

In [ ]:
## 1시부터 24시까지 시간대별 평균 파티 유지 시간을 구합니다.

party_mean_seconds_by_hour = party.pivot_table(values = "party_day_time_total_seconds", index = "acc_id", columns = "party_start_hour", aggfunc = np.mean)
party_mean_seconds_by_hour = party_mean_seconds_by_hour.reset_index(drop = False)
party_mean_seconds_by_hour = party_mean_seconds_by_hour.fillna(0)

## 컬럼 이름을 바꿔줍니다.

rename = ["acc_id"]
for i in range(1,25):
    col_name = "party_mean_seconds_by_hour_" + str(i)
    rename.append(col_name)
party_mean_seconds_by_hour.columns = rename

print(party_mean_seconds_by_hour.shape)
party_mean_seconds_by_hour.head()

### 7-3. 시간대별 Class 나누기

In [ ]:
# 02:00 - 06:00	EarlyMorning (새벽)
# 06:00 - 10:00	Morning (아침)
# 10:00 - 14:00	Noon (오전)
# 14:00 - 18:00	Afternoon (오후)
# 18:00 - 22:00	Evening (저녁)
# 22:00 - 02:00	Night (밤)

In [ ]:
## 시간대 class별 총 파티 유지 시간을 구합니다.

party_total_seconds_by_hour["party_EarlyMorning_total_seconds"] = party_total_seconds_by_hour[["party_total_seconds_by_hour_2", "party_total_seconds_by_hour_3", "party_total_seconds_by_hour_4", "party_total_seconds_by_hour_5"]].T.sum()
party_total_seconds_by_hour["party_Morning_total_seconds"] = party_total_seconds_by_hour[["party_total_seconds_by_hour_6", "party_total_seconds_by_hour_7", "party_total_seconds_by_hour_8", "party_total_seconds_by_hour_9"]].T.sum()
party_total_seconds_by_hour["party_Noon_total_seconds"] = party_total_seconds_by_hour[["party_total_seconds_by_hour_10", "party_total_seconds_by_hour_11", "party_total_seconds_by_hour_12", "party_total_seconds_by_hour_13"]].T.sum()
party_total_seconds_by_hour["party_Afternoon_total_seconds"] = party_total_seconds_by_hour[["party_total_seconds_by_hour_14", "party_total_seconds_by_hour_15", "party_total_seconds_by_hour_16", "party_total_seconds_by_hour_17"]].T.sum()
party_total_seconds_by_hour["party_Evening_total_seconds"] = party_total_seconds_by_hour[["party_total_seconds_by_hour_18", "party_total_seconds_by_hour_19", "party_total_seconds_by_hour_20", "party_total_seconds_by_hour_21"]].T.sum()
party_total_seconds_by_hour["party_Night_total_seconds"] = party_total_seconds_by_hour[["party_total_seconds_by_hour_22", "party_total_seconds_by_hour_23", "party_total_seconds_by_hour_24", "party_total_seconds_by_hour_1"]].T.sum()

print(party_total_seconds_by_hour.shape)
party_total_seconds_by_hour.head()

In [ ]:
## 시간대 class별 평균 파티 유지 시간을 구합니다.

party_mean_seconds_by_hour["party_EarlyMorning_mean_seconds"] = party_mean_seconds_by_hour[["party_mean_seconds_by_hour_2", "party_mean_seconds_by_hour_3", "party_mean_seconds_by_hour_4", "party_mean_seconds_by_hour_5"]].T.mean()
party_mean_seconds_by_hour["party_Morning_mean_seconds"] = party_mean_seconds_by_hour[["party_mean_seconds_by_hour_6", "party_mean_seconds_by_hour_7", "party_mean_seconds_by_hour_8", "party_mean_seconds_by_hour_9"]].T.mean()
party_mean_seconds_by_hour["party_Noon_mean_seconds"] = party_mean_seconds_by_hour[["party_mean_seconds_by_hour_10", "party_mean_seconds_by_hour_11", "party_mean_seconds_by_hour_12", "party_mean_seconds_by_hour_13"]].T.mean()
party_mean_seconds_by_hour["party_Afternoon_mean_seconds"] = party_mean_seconds_by_hour[["party_mean_seconds_by_hour_14", "party_mean_seconds_by_hour_15", "party_mean_seconds_by_hour_16", "party_mean_seconds_by_hour_17"]].T.mean()
party_mean_seconds_by_hour["party_Evening_mean_seconds"] = party_mean_seconds_by_hour[["party_mean_seconds_by_hour_18", "party_mean_seconds_by_hour_19", "party_mean_seconds_by_hour_20", "party_mean_seconds_by_hour_21"]].T.mean()
party_mean_seconds_by_hour["party_Night_mean_seconds"] = party_mean_seconds_by_hour[["party_mean_seconds_by_hour_22", "party_mean_seconds_by_hour_23", "party_mean_seconds_by_hour_24", "party_mean_seconds_by_hour_1"]].T.mean()

print(party_mean_seconds_by_hour.shape)
party_mean_seconds_by_hour.head()

## 8. 파티 멤버수에 따른 분류

In [ ]:
## 파티 멤버 수에 따라 1명, 6명, 12명, 24명 이상, 그 밖의 수 등 5개의 class로 나눕니다.

party.loc[party["member_size"] == 1, "member_size_class"] = "party_member_solo"
party.loc[party["member_size"] == 6, "member_size_class"] = "party_member_six"
party.loc[party["member_size"] == 12, "member_size_class"] = "party_member_twelve"
party.loc[party["member_size"] > 24, "member_size_class"] = "party_member_big"
party.loc[party["member_size_class"].isin(["party_member_solo", "party_member_six", "party_member_twelve", "party_member_big"]) == False, "member_size_class"] = "party_member_irregular"

In [ ]:
total_by_member_class = party.pivot_table("party_day_time_total_seconds", index = "acc_id", columns = "member_size_class", aggfunc = np.sum).fillna(0)

In [ ]:
total_sum = party.pivot_table("party_day_time_total_seconds", index = "acc_id", aggfunc= np.sum)

In [ ]:
total_by_member_class["total"] = total_sum["party_day_time_total_seconds"]

In [ ]:
total_by_member_class["party_member_solo_ratio"] = total_by_member_class["party_member_solo"] / total_by_member_class["total"]
total_by_member_class["party_member_six_ratio"] = total_by_member_class["party_member_six"] / total_by_member_class["total"]
total_by_member_class["party_member_twelve_ratio"] = total_by_member_class["party_member_twelve"] / total_by_member_class["total"]
total_by_member_class["party_member_big_ratio"] = total_by_member_class["party_member_big"] / total_by_member_class["total"]
total_by_member_class["party_member_irregular_ratio"] = total_by_member_class["party_member_irregular"] / total_by_member_class["total"]

In [ ]:
## acc_id를 column으로 변경합니다.
del(total_by_member_class["total"])
total_by_member_class = total_by_member_class.reset_index(drop = False)

In [ ]:
mem = party.pivot_table("member_size", index = "acc_id", aggfunc = np.mean)
total_by_member_class = pd.merge(total_by_member_class, mem, on = "acc_id")

print(total_by_member_class.shape)
total_by_member_class.head()

In [ ]:
total_by_member_class = total_by_member_class[['acc_id', 'party_member_solo', 'party_member_six', 'party_member_twelve', 'party_member_big', 'party_member_irregular',
'party_member_solo_ratio', 'party_member_six_ratio', 'party_member_twelve_ratio', 'party_member_big_ratio', 
'party_member_irregular_ratio', 'member_size']]

In [ ]:
total_by_member_class.rename(columns={'member_size':'mean_member_size'}, inplace=True)

In [ ]:
print(total_by_member_class.shape)
total_by_member_class.head()

## 9. Features & Label

In [ ]:
from functools import reduce

In [ ]:
train_label.head()

In [ ]:
## 추출한 feature들을 하나의 데이터 프레임으로 만듭니다.

dfs = [train_label,df_total, party_by_re_day, party_by_day_of_week, party_re_day_diff_ratio,
       party_count_by_time_class, party_seconds_by_time_class, party_mean_by_time_class,
       party_total_seconds_by_hour, party_mean_seconds_by_hour, total_by_member_class]

df_merge = df_weekday_weekend

In [ ]:
party_final = reduce(lambda left, right: pd.merge(left,right,on='acc_id'), dfs)
print(party_final.shape)
party_final.head()

In [ ]:
party_final = pd.merge(party_final, df_merge, how='outer', on='acc_id')

In [ ]:
print(party_final.shape)
party_final.head()

## 10. To_csv

In [ ]:
party_final.to_csv("party_train.csv")